### **Advanced SQL**

<font color="red">File access required:</font> In Colab this notebook requires first uploading files **Cities.csv**, **Countries.csv**, **Players.csv**, and **Teams.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure these files are in the same workspace as the notebook.

In [ ]:
!pip install prettytable==0.7.2
!pip install ipython-sql

  Preparing metadata (setup.py) ... done
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13695 sha256=61a9ecfbf91beb6ab2a3df1dd3fd728c23cb777f79f02c8a19a9a038fe5f7d91
  Stored in directory: /root/.cache/pip/wheels/ca/f9/66/1ebeb8cdff2211eebb6fce02957f9e0a9ae3da4b7e65512d1b
Successfully built prettytable
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.17.0
    Uninstalling prettytable-3.17.0:
      Successfully uninstalled prettytable-3.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00


In [ ]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [ ]:
# Create database tables from CSV files
with open('Cities.csv') as f: Cities = pd.read_csv(f, index_col=0)
%sql drop table if exists Cities;
%sql --persist Cities
with open('Countries.csv') as f: Countries = pd.read_csv(f, index_col=0)
%sql drop table if exists Countries;
%sql --persist Countries

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted countries'

#### Look at sample of Cities and Countries tables

In [ ]:
%%sql
select * from Cities limit 5

 * sqlite://
Done.


city,country,latitude,longitude,temperature
Aalborg,Denmark,57.03,9.92,7.52
Aberdeen,United Kingdom,57.17,-2.08,8.1
Abisko,Sweden,63.35,18.83,0.2
Adana,Turkey,36.99,35.32,18.67
Albacete,Spain,39.0,-1.87,12.62


In [ ]:
%%sql
select * from Countries limit 5

 * sqlite://
Done.


country,population,EU,coastline
Albania,2.9,no,yes
Andorra,0.07,no,no
Austria,8.57,yes,no
Belarus,9.48,no,no
Belgium,11.37,yes,yes


### Duplicates, table variables

*Warm-up: Find all cities in the EU with temperature > 15*

In [ ]:
%%sql
select city
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes' and temperature > 15

 * sqlite://
Done.


city
Algeciras
Athens
Badajoz
Barcelona
Bari
Cartagena
Catania
Cosenza
Granada
Huelva


*Modify previous query to return EU countries that have a city with temperature > 15, remove duplicates*

*Find number of countries that have a city with latitude > 60 (start with country list then fix)*

In [ ]:
%%sql
select count(country) as count
from Cities
where latitude > 60

 * sqlite://
Done.


count
9


*Modify first query to use table variables*

In [ ]:
%%sql
select city
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes' and temperature > 15

 * sqlite://
Done.


city
Algeciras
Athens
Badajoz
Barcelona
Bari
Cartagena
Catania
Cosenza
Granada
Huelva


*Find all pairs of cities with the same longitude; return the city pairs and their (shared) longitude - notice what's wrong and fix it*

In [ ]:
%%sql
select C1.city, C2.city, C1.longitude
from Cities C1, Cities C2
where C1.longitude = C2.longitude
and C1.city < C2.city

 * sqlite://
Done.


city,city_1,longitude
Burgos,Madrid,-3.68
Craiova,Hrodna,23.83
Maastricht,Stavanger,5.68


*Find all pairs of cities that are near each other, i.e., longitude and latitude are both less than 0.5 apart; return city pairs*

In [ ]:
%%sql
select C1.city, C2.city
from Cities C1, Cities C2
where abs(C1.longitude - C2.longitude) < .5
and abs(C1.latitude - C2.latitude) < .5
and C1.city < C2.city

 * sqlite://
Done.


city,city_1
Ancona,Sarajevo
Basel,Freiburg
Basel,Mulhouse
Bergamo,Milan
Cartagena,Murcia
Heidelberg,Karlsruhe
Horlivka,Makiyivka


### <font color = 'green'>**Your Turn**</font>

*Find all pairs of cities with the same temperature. Return the city pairs along with their shared temperature. What do you think about the data after seeing the answer?*

In [ ]:
%%sql
select C1.city, C2.city, round(C1.temperature,2) as temperature
from Cities C1, Cities C2
where C1.temperature = C2.temperature
and C1.city < C2.city

 * sqlite://
Done.


city,city_1,temperature
Basel,Zurich,7.35
Bonn,Maastricht,10.44
Bratislava,Gyor,10.62
Brno,Vienna,8.65
Brugge,Lille,11.68
Bucharest,Le Mans,12.81
Bucharest,Ruse,12.81
Craiova,Pleven,12.72
Daugavpils,Vilnius,6.51
Edinburgh,Kremenchuk,8.99


### Subqueries in Where clause

*Find all countries in the Countries table with no city in the Cities table*

In [ ]:
%%sql
select country
from Countries
where not exists (

  select * from Cities
  where Cities.country = Countries.country

)

 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg
Turkey


*Find countries in the EU that have a city with temperature > 15*

In [ ]:
%%sql
select country
from Countries
where EU = 'yes'
and exists (select * from Cities
            where Cities.country = Countries.country
            and temperature > 15)

 * sqlite://
Done.


country
Croatia
France
Greece
Italy
Portugal
Spain


*Find number of countries that have a city with latitude > 60 (start with country list)*

In [ ]:
%%sql
select count(country) as count
from Countries
where exists (select * from Cities
              where Cities.country = Countries.country
              and latitude > 60)

 * sqlite://
Done.


count
3


*Find the westernmost city; return the city and longitude*

In [ ]:
%%sql
select city, longitude
from Cities C1
where not exists (select * from Cities C2
                  where C2.longitude < C1.longitude)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


*Add easternmost to previous query*

In [ ]:
%%sql
select city, longitude
from Cities C1
where not exists (select * from Cities C2
                  where C2.longitude > C1.longitude)

 * sqlite://
Done.


city,longitude
Horlivka,38.05


*Westernmost city query using = and min*

In [ ]:
%%sql
select city, longitude
from Cities
where longitude = (select min(longitude) from Cities)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


*Find all cities whose temperature is more than 50% higher than the average; return the city, country, and temperature, ordered by descending temperature*

In [ ]:
%%sql
select city, country, round(temperature,2) as temperature
from Cities
where temperature > (select avg(temperature) * 1.5 from Cities)
order by temperature desc

 * sqlite://
Done.


city,country,temperature
Palermo,Italy,21.66
Athens,Greece,21.07
Algeciras,Spain,21.03
Cartagena,Spain,20.96
Kalamata,Greece,20.93
Marbella,Spain,20.8
Huelva,Spain,20.68
Patras,Greece,20.45
Cosenza,Italy,20.09
Messina,Italy,20.01


*Number of cities in the EU*

In [ ]:
%%sql
select count() as count
from Cities
where country in (select country from Countries where EU = 'yes')

 * sqlite://
Done.


count
150


*Modify previous query to use "not in"*

In [ ]:
%%sql
select count() as count
from Cities
where country not in (select country from Countries where EU = 'yes')

 * sqlite://
Done.


count
39


*Same query using join instead of subquery*

In [ ]:
%%sql
select count() as count
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes'

 * sqlite://
Done.


count
150


*Number of countries with no coastline and a city with longitude < 20*

In [ ]:
%%sql
select count() as countries
from Countries
where coastline = 'no'
and exists (select * from Cities where country = Countries.country
            and longitude < 20)

 * sqlite://
Done.


countries
7


*Same query using join instead of subquery (see what's wrong and fix it)*

In [ ]:
%%sql
select count(distinct Countries.country) as countries
from Countries
join Cities on Countries.country = Cities.country
where Countries.coastline = 'no' and Cities.longitude < 20

 * sqlite://
Done.


countries
7


*Find countries in Countries table with no city in Cities table using join instead of subquery (subquery version repeated first)*

In [ ]:
%%sql
select country
from Countries
where not exists (select * from Cities
                  where Cities.country = Countries.country)

 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg
Turkey


In [ ]:
%%sql
FILL IN

 * sqlite://
(sqlite3.OperationalError) near "FILL": syntax error
[SQL: FILL IN]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### <font color = 'green'>**Your Turn**</font>

*Find all cities in a country whose population is < 2; return the city and country. First write the query without a subquery.*

In [ ]:
%%sql
select Cities.city, Countries.country
from Cities
join Countries on Cities.country = Countries.country
where Countries.population < 2

 * sqlite://
Done.


city,country
Andorra,Andorra
Daugavpils,Latvia
Podgorica,Montenegro
Riga,Latvia
Tallinn,Estonia
Tartu,Estonia


*Now write the same query using a subquery instead of a join.*

In [ ]:
%%sql
select city, country
from Cities
where country in (select country from Countries where population < 2)

 * sqlite://
Done.


city,country
Andorra,Andorra
Daugavpils,Latvia
Podgorica,Montenegro
Riga,Latvia
Tallinn,Estonia
Tartu,Estonia


*Find all countries with no city having a temperature > 6*

In [ ]:
%%sql
select country
from Countries
where not exists (select * from Cities
                  where Cities.country = Countries.country
                  and temperature > 6)

 * sqlite://
Done.


country
Cyprus
Estonia
Finland
Iceland
Kosovo
Liechtenstein
Luxembourg
Turkey


*Now try to write the same query without a subquery; can you?*

In [ ]:
%%sql
select Countries.country
from Countries
left join (select distinct country from Cities where temperature > 6) as Cities
on Countries.country = Cities.country
where Cities.country is null;

 * sqlite://
Done.


country
Cyprus
Estonia
Finland
Iceland
Kosovo
Liechtenstein
Luxembourg
Turkey


### Aggregation with Having clause

*Find all countries with average city temperature > 10; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > 10

 * sqlite://
Done.


country,avg(temperature)
Albania,18.367800000000003
Andorra,10.56
Belgium,11.676500000000003
Bosnia and Herzegovina,11.616000000000001
Bulgaria,12.6324
Croatia,13.146650000000001
France,12.282844444444445
Greece,20.452025000000003
Hungary,10.562750000000001
Ireland,11.253000000000002


*Find all countries with more than 5 cities above latitude 50*

In [ ]:
%%sql
select country
from Cities
where latitude > 50
group by country
having count() > 5

 * sqlite://
Done.


country
Belarus
Germany
Poland
Sweden
United Kingdom


*Same query without Having clause*

In [ ]:
%%sql
select distinct country
from Cities C1
where 5 < (select count() from Cities C2
           where C1.country=C2.country
           and latitude > 50)

 * sqlite://
Done.


country
United Kingdom
Sweden
Germany
Poland
Belarus


*Which combinations of EU versus non-EU and coastline versus no-coastline have a minimum population greater than 0.5?*

In [ ]:
%%sql
select EU, coastline, min(population)
from Countries
group by EU, coastline
having min(population) > 0.5

 * sqlite://
Done.


EU,coastline,min(population)
yes,no,0.58
yes,yes,1.18


*Find all countries with average city temperature more than 50% higher than the overall average; return country and average temperature*

In [ ]:
%%sql
select country, avg(temperature)
from Cities
group by country
having avg(temperature) > (select 1.5 * avg(temperature) from Cities)

 * sqlite://
Done.


country,avg(temperature)
Albania,18.367800000000003
Greece,20.452025000000003
Portugal,17.5087
Spain,17.228383333333333


### <font color = 'green'>**Your Turn**</font>

*Find all countries whose average city longitude is lower than the overall average longitude, and whose average city latitude is higher than the overall average latitude. Return the countries. Note: Yes, you can use "and" in Having clauses!*

In [ ]:
%%sql
select country
from Cities
group by country
having avg(longitude) < (select avg(longitude) from Cities)
and avg(latitude) > (select avg(latitude) from Cities)

 * sqlite://
Done.


country
Belgium
Denmark
Germany
Ireland
Netherlands
Norway
United Kingdom


### Subqueries in From and Select clauses

*Find all countries with both cold and warm cities -- at least one city with temperature < 9 and one city with temperature > 14*

In [ ]:
%%sql
select distinct C1.country
from Cities C1, Cities C2
where C1.country = C2.country
and C1.temperature < 9 and C2.temperature > 14

 * sqlite://
Done.


country
Italy
France


*Modify query to also return count of cold and warm cities (then show without column renaming)*

In [ ]:
%%sql
select C1.country, count() as numcold, count() as numwarm
from Cities C1, Cities C2
where C1.country = C2.country
and C1.temperature < 9 and C2.temperature > 14
group by C1.country

 * sqlite://
Done.


country,numcold,numwarm
France,2,2
Italy,12,12


In [ ]:
# (select count() from Cities where country = C1.country and temperature < 9) as numcold,
# (select count() from Cities where country = C1.country and temperature > 14) as numwarm

*Same query using subquery in From clause instead of Select clause*

In [ ]:
%%sql
select Cold.country, numcold, numwarm
from (select country, count() as numcold from Cities
      where temperature < 9 group by country) Cold,
     (select country, count() as numwarm from Cities
      where temperature > 14 group by country) Warm
where Cold.country = Warm.country

 * sqlite://
Done.


country,numcold,numwarm
France,1,2
Italy,1,12


### Data modification

*Increase all city temperatures by 10%*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature

 * sqlite://
189 rows affected.


[]

In [ ]:
%%sql
select avg(temperature) from Cities

 * sqlite://
Done.


avg(temperature)
12.116107724867724


*Increase temperatures another 10% for cities in countries with coastline*

In [ ]:
%%sql
update Cities
set temperature = 1.1 * temperature
where country in (select country from Countries
                  where coastline = 'yes')

 * sqlite://
159 rows affected.


[]

*Delete all cities in Turkey*

In [ ]:
%%sql
delete from Cities
where country = 'Turkey'

 * sqlite://
0 rows affected.


[]

*Create a new table NonEU containing list of cities (with country) not in the EU*

In [ ]:
%%sql
drop table if exists NonEU;
create table NonEU(city, country);
insert into NonEU
  select city, country from cities
  where country in (select country from Countries
                    where EU = 'no');
select * from NonEU

 * sqlite://
Done.
Done.
39 rows affected.
Done.


city,country
Andorra,Andorra
Balti,Moldova
Basel,Switzerland
Belgrade,Serbia
Bergen,Norway
Bila Tserkva,Ukraine
Bodo,Norway
Brest,Belarus
Cherkasy,Ukraine
Chernihiv,Ukraine


*Add your city*

In [ ]:
%%sql
insert into NonEU values ('Milan','Italy');
select * from NonEU

 * sqlite://
1 rows affected.
Done.


city,country
Andorra,Andorra
Balti,Moldova
Basel,Switzerland
Belgrade,Serbia
Bergen,Norway
Bila Tserkva,Ukraine
Bodo,Norway
Brest,Belarus
Cherkasy,Ukraine
Chernihiv,Ukraine


### <font color = 'green'>**Your Turn - Advanced SQL on World Cup Data**</font>

In [ ]:
# Create database tables from CSV files
with open('Players.csv') as f: Players = pd.read_csv(f, index_col=0)
%sql drop table if exists Players;
%sql --persist Players
with open('Teams.csv') as f: Teams = pd.read_csv(f, index_col=0)
%sql drop table if exists Teams;
%sql --persist Teams

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted teams'

#### Look at sample of Players and Teams tables

In [ ]:
%%sql
select * from Players limit 5

 * sqlite://
Done.


surname,team,position,minutes,shots,passes,tackles,saves
Abdoun,Algeria,midfielder,16,0,6,0,0
Belhadj,Algeria,defender,270,1,146,8,0
Boudebouz,Algeria,midfielder,74,3,28,1,0
Bougherra,Algeria,defender,270,1,89,11,0
Chaouchi,Algeria,goalkeeper,90,0,17,0,2


In [ ]:
%%sql
select * from Teams limit 5

 * sqlite://
Done.


team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
Brazil,1,5,3,1,1,9,4,7,2
Spain,2,6,5,0,1,7,2,3,0
Portugal,3,4,1,2,1,7,1,8,1
Netherlands,4,6,6,0,0,12,5,15,0
Italy,5,3,0,2,1,4,5,5,0


*1) Find all pairs of teams who have the same number of goalsFor as
each other and the same number of goalsAgainst as each other.
Return the teams and numbers of goalsFor and goalsAgainst.
Make sure to return each pair only once.*

In [ ]:
%%sql
select T1.team, T2.team, T1.goalsFor, T2.goalsAgainst
from Teams T1, Teams T2
where T1.team < T2.team
and T1.goalsFor = T2.goalsFor
and T1.goalsAgainst = T2.goalsAgainst

 * sqlite://
Done.


team,team_1,goalsFor,goalsAgainst
Italy,Mexico,4,5
England,Nigeria,3,5
England,South Africa,3,5
Chile,England,3,5
Chile,Nigeria,3,5
Chile,South Africa,3,5
Cameroon,Greece,2,5
Australia,Denmark,3,6
Nigeria,South Africa,3,5


*2) Find all teams with ranking <30 where no player made more than 150 passes. Return the team and ranking.*

In [ ]:
%%sql
select team, ranking
from Teams
where ranking < 30
and not exists (select * from Players
                where Players.team = Teams.team
                and passes > 150)

 * sqlite://
Done.


team,ranking
France,9
Nigeria,21
Switzerland,24


*3) Which players made more shots than 5x the overall average number of shots? Return the player surname, position, and team.*

In [ ]:
%%sql
select surname, position, team
from Players
where shots > 5 * (select avg(shots) from Players)

 * sqlite://
Done.


surname,position,team
Higuain,forward,Argentina
Messi,forward,Argentina
Podolski,forward,Germany
Boateng,midfielder,Ghana
Gyan,forward,Ghana
Sneijder,midfielder,Netherlands
van Persie,forward,Netherlands
Jong Tae-Se,forward,North Korea
Ronaldo,forward,Portugal
Park Chu-Young,forward,South Korea


*4) Find all team-position pairs where the average number of passes made by players in that position on that team is greater than 150. Return the team-position pairs.*

In [ ]:
%%sql
select team, position
from Players
group by team, position
having avg(passes) > 150

 * sqlite://
Done.


team,position
Argentina,midfielder
Brazil,defender
Germany,defender
Germany,midfielder
Ghana,midfielder
Mexico,defender
Netherlands,defender
Netherlands,midfielder
Spain,defender
Spain,midfielder


*5) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [ ]:
%%sql
select team, avg(passes)
from Players
where position = 'defender'
group by team
having avg(passes) > 150
order by avg(passes) desc

 * sqlite://
Done.


team,avg(passes)
Spain,213.0
Brazil,190.0
Germany,189.83333333333334
Netherlands,182.5
Mexico,152.14285714285714


### <font color = 'green'>**Your Turn Extra - Advanced SQL on Titanic Data**</font>

<font color="red">File access required:</font> In Colab these extra problems require first uploading **Titanic.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure this file is in the same workspace as the notebook.

In [ ]:
# Load dataabase table from CSV file
with open('Titanic.csv') as f: Titanic = pd.read_csv(f, index_col=0)
%sql drop table if exists Titanic;
%sql --persist Titanic

 * sqlite://
Done.
 * sqlite://


'Persisted titanic'

#### Look at sample of Titanic table

In [ ]:
%%sql
select * from Titanic limit 5

 * sqlite://
Done.


last,first,gender,age,class,fare,embarked,survived
Abbing,Mr. Anthony,M,42.0,3,7.55,Southampton,no
Abbott,Mrs. Stanton (Rosa Hunt),F,35.0,3,20.25,Southampton,yes
Abbott,Mr. Rossmore Edward,M,16.0,3,20.25,Southampton,no
Abelson,Mr. Samuel,M,30.0,2,24.0,Cherbourg,no
Abelson,Mrs. Samuel (Hannah Wizosky),F,28.0,2,24.0,Cherbourg,yes


*1) Find pairs of passengers who are likely to be twin children: same last name, same age, same embarkation, and age is under 18. Return each pair once, including their last name, first names, age, and embarkation city.*

In [ ]:
%%sql
select T1.last, T1.first, T1.age, T1.embarked, T2.last, T2.first, T2.age, T2.embarked
from Titanic T1, Titanic T2
where T1.last = T2.last
and T1.age = T2.age
and T1.embarked = T2.embarked
and T1.age < 18
and T2.age < 18
and T1.first < T2.first

 * sqlite://
Done.


last,first,age,embarked,last_1,first_1,age_1,embarked_1
Baclini,Miss Eugenie,0.75,Cherbourg,Baclini,Miss Helene Barbara,0.75,Cherbourg
Calic,Mr. Jovo,17.0,Southampton,Calic,Mr. Petar,17.0,Southampton


*2) Which embarkation cities have more than 40 passengers whose age is missing? Reminder: Blanks in SQL tables are given a special value called 'null', and conditions 'A is null' and 'A is not null' can be used in Where clauses to check whether attribute A has the 'null' value.*

In [ ]:
%%sql
select embarked
from Titanic
where age is null
group by embarked
having count() > 40

 * sqlite://
Done.


embarked
Queenstown
Southampton


*3) Find all classes where the average fare paid by passengers in that class was more than twice the overall average or less than half the overall average.*

In [ ]:
%%sql
select class
from Titanic
group by class
having avg(fare) > 2 * (select avg(fare) from Titanic)
or avg(fare) < 0.5 * (select avg(fare) from Titanic)

 * sqlite://
Done.


class
1
3


*4) What is the average number of passengers per last name? Hint: Requires using a subquery in the From clause*

In [ ]:
%%sql
select round(avg(num),2) as avg_num
from (select last, count() as num
      from Titanic
      group by last)

 * sqlite://
Done.


avg_num
1.34


*5) List each class and its survival rate, i.e., the fraction of passengers in that class who survived. Hints: Use subqueries in the From clause to compute the number of survivers per class and total passengers per class, and force floating point division by multiplying one operand by 1.0*

In [ ]:
%%sql
select T1.class, (1.0 * survived) / total as survival_rate
from (select class, count() as survived
      from Titanic
      where survived = 'yes'
      group by class) T1, (select class, count() as total
                             from Titanic
                             group by class) T2
where T1.class = T2.class

 * sqlite://
Done.


class,survival_rate
1,0.6296296296296297
2,0.47282608695652173
3,0.24236252545824846


*6) Modify your previous query to return the survival rate by gender, i.e., of females and of males.*

In [ ]:
%%sql
select T1.gender, (1.0 * survived) / total as survival_rate
from (select gender, count() as survived
      from Titanic
      where survived = 'yes'
      group by gender) T1, (select gender, count() as total
                             from Titanic
                             group by gender) T2
where T1.gender = T2.gender

 * sqlite://
Done.


gender,survival_rate
F,0.7420382165605095
M,0.18890814558058924


*7) Now return the survival rate of children versus adults, i.e., of passengers under age 18 versus those 18 or over (ignoring passengers whose age is missing).*

In [ ]:
%%sql
select T1.age_group, (1.0 * survived) / total as survival_rate
from (select
        case when age < 18 then 'child'
        else 'adult' end as age_group,
        count() as survived
      from Titanic
      where survived = 'yes' and age is not null
      group by age_group) T1,
     (select
        case when age < 18 then 'child'
        else 'adult' end as age_group,
        count() as total
      from Titanic
      where age is not null
      group by age_group) T2
where T1.age_group = T2.age_group

 * sqlite://
Done.


age_group,survival_rate
adult,0.3810316139767055
child,0.5398230088495575


### <font color = 'green'>**Your Turn Extra - SQL Data Modification on Titanic Data**</font>

In [ ]:
# Reload table from CSV file
# NOTE: You may want to reload frequently to reset the data as you
# experiment with modifications
with open('Titanic.csv') as f: Titanic = pd.read_csv(f, index_col=0)
%sql drop table if exists Titanic;
%sql persist Titanic

 * sqlite://
Done.
 * sqlite://
(sqlite3.OperationalError) near "persist": syntax error
[SQL: persist Titanic]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


*1) Subtract 5 from the fare paid by any passenger under the age of 10. Then compute the new average fare. NOTE: You can put multiple SQL statements in one cell separated by a semicolon.*

In [ ]:
%%sql
select avg(fare)
from (select fare - 5 as fare
      from Titanic
      where age < 10)

 * sqlite://
Done.


avg(fare)
25.5783870967742


*2) Create a new table called Survivors, containing the last and first names of all passengers who survived. Then count the number of tuples in the new table.*

In [ ]:
%%sql
create table Survivors as
select last, first
from Titanic
where survived = 'yes'

 * sqlite://
Done.


[]

In [ ]:
%%sql
select count() as num_survivors
from Survivors

 * sqlite://
Done.


num_survivors
342


*3) In the Titanic table delete all but the highest-paying passengers.*

In [ ]:
%%sql
delete from Titanic
where fare not in (select max(fare) from Titanic)

 * sqlite://
888 rows affected.


[]

*4) In what's left of the table after (3), insert a new tuple for yourself. You can decide your class, fare, where you embarked, and whether you survived. Then show the whole table.*

In [ ]:
%%sql
insert into Titanic values ('Goat','Mr. CB', 'F', 22, 1, 600.00, 'Tokyo', 'yes');
select * from Titanic

 * sqlite://
1 rows affected.
Done.


last,first,gender,age,class,fare,embarked,survived
Cardeza,Mr. Thomas Drake Martinez,M,36.0,1,512.33,Cherbourg,yes
Lesurer,Mr. Gustave J,M,35.0,1,512.33,Cherbourg,yes
Ward,Miss Anna,F,35.0,1,512.33,Cherbourg,yes
Goat,Mr. CB,F,22.0,1,600.0,Tokyo,yes
